# 🤖 Tool-Calling Agents in LangChain

## Learning Objectives
In this notebook, you will learn:
1. **What is an Agent?** - An LLM that can autonomously decide which tools to use
2. **Basic Tool Calling** - Binding tools to an LLM and processing tool calls
3. **Complete Agents** - Using `AgentExecutor` for automatic tool execution
4. **Pydantic Schemas** - Structured input validation for tools
5. **Parallel Tool Calling** - Handling multiple tool calls in a single request

## Key Concepts
- **Tool Binding**: Attaching tools to an LLM so it knows what's available
- **Tool Calls**: The LLM's request to execute a specific tool with arguments
- **Agent Loop**: The cycle of LLM thinking → tool execution → processing results
- **AgentExecutor**: LangChain's built-in agent runner

## Prerequisites
- Completed the previous notebook on Tools in LangChain
- Basic understanding of LLMs and prompts

---
## 🔧 Step 1: Environment Setup

Let's set up our LLM using helper functions that manage API keys and configuration.

In [ ]:
# ============================================================================
# SETUP: Import LLM Helper Functions & Initialize LLM
# ============================================================================
# We use helper functions to create LLM instances with proper configuration.
# These functions handle API key loading from .env and model configuration.

import os
import sys
import platform

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference with open-source models)
# - get_openai_llm(): Creates an OpenAI GPT model
# - get_databricks_llm(): Creates a Databricks-hosted LLM
from helpers.utils import get_groq_llm, get_openai_llm, get_databricks_llm

print("✅ LLM helpers imported successfully!")
print(f"📍 Running on: {platform.system()}")

# -----------------------------------------------------------------------------
# Initialize the LLM based on platform or preference
# The choice of LLM affects tool calling capabilities and speed
# -----------------------------------------------------------------------------
if sys.platform == "win32":
    # Windows: Use Groq for fast inference
    llm = get_groq_llm()
elif sys.platform == "darwin":
    # macOS: Use Databricks-hosted Gemini
    llm = get_databricks_llm("databricks-gemini-2-5-pro")  
else:
    # Linux: Default to Groq
    llm = get_groq_llm()

# Print which LLM we're using
if hasattr(llm, 'model_name'):
    print(f"🤖 LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"🤖 LLM initialized: {llm.model} (Databricks)")
else:
    print("🤖 LLM initialized successfully")

---
## 🔗 Part 1: Basic Tool Calling

### How Tool Calling Works
1. **Define Tools**: Create functions with descriptive docstrings
2. **Bind Tools to LLM**: Use `llm.bind_tools()` to attach tools
3. **Invoke LLM**: Send a query that might need tool usage
4. **Process Tool Calls**: LLM returns `tool_calls` instead of (or alongside) text

> **Important**: The LLM doesn't execute tools - it only generates the tool call request!
> Your code is responsible for actually running the tools.

In [ ]:
# ============================================================================
# BASIC TOOL CALLING: Define and Bind Tools
# ============================================================================
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# -----------------------------------------------------------------------------
# Define Custom Tools
# Note: Docstrings are CRITICAL - the LLM uses them to understand when to use each tool
# -----------------------------------------------------------------------------

@tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city.
    
    Args:
        city: Name of the city (e.g., 'Bangalore', 'Mumbai', 'Delhi')
    
    Returns:
        A string with temperature and weather conditions
    """
    # This is simulated data - in production, you'd call a real weather API
    weather_data = {
        "bangalore": "28°C, Partly Cloudy",
        "mumbai": "32°C, Humid",
        "delhi": "25°C, Sunny"
    }
    return weather_data.get(city.lower(), "Weather data not available for this city")

@tool
def calculate(expression: str) -> str:
    """
    Evaluate a mathematical expression and return the result.
    
    Args:
        expression: A valid Python mathematical expression (e.g., '25 * 4', '100 / 5 + 10')
    
    Returns:
        The calculated result as a string
    """
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# -----------------------------------------------------------------------------
# Bind Tools to the LLM
# This creates a new LLM instance that knows about our tools
# -----------------------------------------------------------------------------
tools = [get_weather, calculate]
llm_with_tools = llm.bind_tools(tools)

print("✅ Tools bound to LLM:")
for t in tools:
    print(f"   📌 {t.name}: {t.description[:60]}...")

In [ ]:
# ============================================================================
# BASIC TOOL CALLING: Demonstration
# ============================================================================
# When we invoke the LLM with a query that needs a tool, it returns tool_calls

# Ask a question that requires the weather tool
response = llm_with_tools.invoke("What's the weather in Bangalore?")

print("🤖 LLM Response Analysis:")
print("=" * 60)
print(f"\n📄 Content (if any): {response.content or '(No text content - tool call generated)'}") 
print(f"\n🔧 Tool Calls:")
for tc in response.tool_calls:
    print(f"   - Tool: {tc['name']}")
    print(f"   - Arguments: {tc['args']}")
    print(f"   - ID: {tc['id']}")

# Note: At this point, the tool hasn't been executed yet!
# We need to execute it ourselves using the arguments provided

---
## 🚀 Part 2: Complete Agent with AgentExecutor

While manually processing tool calls works, it's tedious. LangChain provides **AgentExecutor** which:
1. Automatically executes tool calls
2. Feeds results back to the LLM
3. Continues until the LLM provides a final answer
4. Handles multi-step reasoning

### The Agent Loop
```
User Query → LLM → Tool Call? → Execute Tool → Feed Result → LLM → ... → Final Answer
```

In [ ]:
# ============================================================================
# COMPLETE AGENT: Multi-Tool Agent with AgentExecutor
# ============================================================================
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# -----------------------------------------------------------------------------
# Define a Set of Diverse Tools
# This demonstrates how an agent can work with multiple tools
# -----------------------------------------------------------------------------

@tool
def search_database(query: str) -> str:
    """
    Search the internal database for information about a topic.
    Use this for internal company data, product information, or customer records.
    """
    # Simulated database search
    return f"Found 3 results for: {query}"

@tool
def get_stock_price(symbol: str) -> str:
    """
    Get the current stock price for a given stock symbol.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'GOOGL', 'MSFT', 'AAPL')
    """
    # Simulated stock prices (in production, use a real API)
    prices = {"GOOGL": 175.50, "MSFT": 420.30, "AAPL": 195.80}
    price = prices.get(symbol.upper(), "Not found")
    return f"{symbol.upper()}: ${price}"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """
    Send an email to the specified recipient.
    
    Args:
        to: Email address of the recipient
        subject: Subject line of the email
        body: Body content of the email
    """
    # Simulated email sending (in production, use SMTP or email API)
    return f"Email sent to {to} with subject: {subject}"

tools = [search_database, get_stock_price, send_email]

# -----------------------------------------------------------------------------
# Create the Prompt Template
# The placeholder {agent_scratchpad} is where LangChain inserts the agent's
# thinking and tool call results during the agent loop
# -----------------------------------------------------------------------------
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the available tools to help answer the user's questions. Always provide helpful and complete responses."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # Critical: holds intermediate steps
])

# -----------------------------------------------------------------------------
# Create and Run the Agent
# -----------------------------------------------------------------------------
# create_tool_calling_agent: Creates an agent that uses native tool calling
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor: Runs the agent loop automatically
# - verbose=True: Shows the thinking process
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True  # Set to False in production
)

# Run a complex multi-step task
print("🚀 Running Agent with Multi-Step Task:")
print("=" * 60)

result = agent_executor.invoke({
    "input": "What's the stock price of Google? Once you get the stock price, send an email to sourav.banerjee@gmail.com with the information."
})

print("\n" + "=" * 60)
print("📤 Final Output:")
print(result["output"])

---
## 📋 Part 3: Structured Tools with Pydantic

For production use, you should define your tool inputs using **Pydantic models**. This provides:
- **Type Safety**: Inputs are validated before tool execution
- **Clear Documentation**: Each field has a description
- **Default Values**: Optional parameters with sensible defaults
- **Better LLM Understanding**: Rich schema helps the LLM generate correct arguments

In [ ]:
# ============================================================================
# STRUCTURED TOOLS: Using Pydantic for Input Validation
# ============================================================================
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# -----------------------------------------------------------------------------
# Define the Input Schema with Pydantic
# This provides type checking, validation, and rich descriptions
# -----------------------------------------------------------------------------
class DataQueryInput(BaseModel):
    """
    Input schema for querying data from a database table.
    All fields must be provided to execute the query.
    """
    table_name: str = Field(
        description="Name of the table to query (e.g., 'customers', 'sales', 'products')"
    )
    filters: dict = Field(
        description="Filter conditions as key-value pairs (e.g., {'region': 'Asia', 'status': 'active'})"
    )
    limit: int = Field(
        default=10, 
        description="Maximum number of rows to return (default: 10)"
    )

# -----------------------------------------------------------------------------
# Create a Tool with the Pydantic Schema
# The args_schema parameter links the tool to our input model
# -----------------------------------------------------------------------------
@tool(args_schema=DataQueryInput)
def query_data(table_name: str, filters: dict, limit: int = 10) -> str:
    """
    Query data from a specified database table with filters.
    
    Returns matching records up to the specified limit.
    """
    # Simulated query (in production, execute actual SQL)
    return f"Queried {table_name} with filters {filters}, returning up to {limit} rows"

print("✅ Structured tool created!")
print(f"📋 Tool Name: {query_data.name}")
print(f"📝 Tool Description: {query_data.description}")
print(f"📦 Tool Arguments Schema: {query_data.args}")

In [ ]:
# ============================================================================
# STRUCTURED TOOLS: Demonstration
# ============================================================================

# Bind the structured tool to LLM
llm_with_structured_tools = llm.bind_tools([query_data])

# Ask a natural language question that should trigger the tool
response = llm_with_structured_tools.invoke(
    "Get the top 5 customers from the sales table where region is Asia"
)

print("🤖 LLM Generated Tool Call:")
print("=" * 60)
if response.tool_calls:
    tc = response.tool_calls[0]
    print(f"📌 Tool: {tc['name']}")
    print(f"📦 Arguments:")
    for key, value in tc['args'].items():
        print(f"   - {key}: {value}")
    
    # Execute the tool with the LLM-generated arguments
    print(f"\n🔧 Tool Execution Result:")
    result = query_data.invoke(tc["args"])
    print(f"   {result}")

---
## ⚡ Part 4: Parallel Tool Calling

Many modern LLMs can call **multiple tools in parallel** when they're independent. This is much faster than sequential tool calling for queries like:
- "Get weather in Bangalore AND Mumbai" (2 weather calls)
- "Get weather in Delhi and calculate 25 * 4" (different tools)

The LLM will return multiple tool calls in a single response.

In [ ]:
# ============================================================================
# PARALLEL TOOL CALLING: Setup
# ============================================================================
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# Use the weather and calculate tools from earlier
tools = [get_weather, calculate]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM ready for parallel tool calling")
print(f"📌 Available tools: {[t.name for t in tools]}")

In [ ]:
# ============================================================================
# PARALLEL TOOL CALLING: Multiple Tools in One Query
# ============================================================================

# Ask a question that requires multiple tools
query = "Get weather in Bangalore and Mumbai, also calculate 25 * 4"

response = llm_with_tools.invoke(query)

print(f"📝 Query: {query}")
print("\n🔧 LLM generated {0} parallel tool calls:".format(len(response.tool_calls)))
print("=" * 60)

for i, tool_call in enumerate(response.tool_calls, 1):
    print(f"\n{i}. Tool: {tool_call['name']}")
    print(f"   Arguments: {tool_call['args']}")

In [ ]:
# ============================================================================
# PARALLEL TOOL CALLING: Complete Agent Execution
# ============================================================================
# AgentExecutor handles parallel tool calls automatically

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create a prompt that encourages parallel tool use
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Call ALL tools needed to fully answer the user's request. When possible, call multiple tools in parallel."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("🚀 Running Agent with Parallel Tool Calls:")
print("=" * 60)

result = agent_executor.invoke({
    "input": "Get weather in Bangalore and Mumbai, also calculate 25 * 4"
})

print("\n" + "=" * 60)
print("📤 Final Output:")
print(result["output"] if "output" in result else result)

---
## 📝 Summary

In this notebook, we learned how to build **Tool-Calling Agents** in LangChain:

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Tool Binding** | `llm.bind_tools(tools)` - Attaches tools to an LLM |
| **Tool Calls** | LLM's request to execute a tool (name + arguments) |
| **AgentExecutor** | Automates the agent loop (tool execution + result handling) |
| **Pydantic Schemas** | Type-safe tool inputs with validation |
| **Parallel Calling** | Multiple independent tools called simultaneously |

### Code Patterns

```python
# 1. Define a tool
@tool
def my_tool(arg: str) -> str:
    """Description for LLM to understand when to use this tool."""
    return "result"

# 2. Bind tools to LLM
llm_with_tools = llm.bind_tools([my_tool])

# 3. Create and run agent
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools)
result = executor.invoke({"input": "..."})
```

### Next Steps
- Explore **LangGraph** for more complex agent workflows
- Learn about **Memory** to build agents that remember context
- Build **Custom Agents** with specialized behaviors